In [1]:
!source activate py36

In [2]:
import sys

sys.path.append('/home/faske/g/anaconda3/envs/py36/lib/python3.6/site-packages')
sys.path.append("/home/faske/g/ipynb/include_utils")

import ipyparallel as ipp
import os, time
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
#import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE, call, check_output
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
#import pysam
from collections import OrderedDict, namedtuple, Counter
import operator
import multiprocessing as mp
import shutil
import tempfile
#from ipyparallel import Client
import scandir
import glob
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import pickle
#import Levenshtein as lv

In [5]:
ent_dir = "/home/faske/g/balsam/entropy"

In [6]:
cd $ent_dir

/working/tfaske/balsam/entropy


In [7]:
vcf = 'good_snps.recode.vcf'
vcf

'good_snps.recode.vcf'

In [8]:
perl_mpgl = '/home/faske/g/perl_scripts/vcf2mpglV1.3TLP.pl'
perl_mean = '/home/faske/g/perl_scripts/gl2genestV1.3.pl'

In [9]:
!perl $perl_mpgl $vcf

Number of loci: 38103; number of individuals 364


In [10]:
mpgl = 'good_snps.recode.mpgl'

In [11]:
!perl $perl_mean $mpgl mean

In [12]:
%load_ext rpy2.ipython

In [14]:
%%R
require(readr)
require(MASS)
require(LEA)
require(ggplot2)

source('/home/faske/g/src/R/Imports.R')

setwd('/home/faske/g/balsam/entropy')

In [15]:
%%R
g <- read.table("pntest_mean_good_snps.recode.txt", header=F)
dim(g)

[1] 38103   364


In [16]:
%%R
Pop_ID <- read.csv("Pop_ID.csv")
pca_df <- PCA_entropy(t(g))

    PC1     PC2     PC3     PC4     PC5 
0.08851 0.04491 0.02174 0.01850 0.01353 


In [17]:
%%R
#######################################################################################
                                    #### LDA ####
#######################################################################################

k2<-kmeans(pca_df[,1:5],2,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k3<-kmeans(pca_df[,1:5],3,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k4<-kmeans(pca_df[,1:5],4,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k5<-kmeans(pca_df[,1:5],5,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k6<-kmeans(pca_df[,1:5],6,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k7<-kmeans(pca_df[,1:5],7,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k8<-kmeans(pca_df[,1:5],8,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k9<-kmeans(pca_df[,1:5],9,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k10<-kmeans(pca_df[,1:5],10,iter.max=10,nstart=10,algorithm="Hartigan-Wong")

ldak2<-lda(x=pca_df[,1:5],grouping=k2$cluster,CV=TRUE)
ldak3<-lda(x=pca_df[,1:5],grouping=k3$cluster,CV=TRUE)
ldak4<-lda(x=pca_df[,1:5],grouping=k4$cluster,CV=TRUE)
ldak5<-lda(x=pca_df[,1:5],grouping=k5$cluster,CV=TRUE)
ldak6<-lda(x=pca_df[,1:5],grouping=k6$cluster,CV=TRUE)
ldak7<-lda(x=pca_df[,1:5],grouping=k7$cluster,CV=TRUE)
ldak8<-lda(x=pca_df[,1:5],grouping=k8$cluster,CV=TRUE)
ldak9<-lda(x=pca_df[,1:5],grouping=k9$cluster,CV=TRUE)
ldak10<-lda(x=pca_df[,1:5],grouping=k10$cluster,CV=TRUE)

write.table(round(ldak2$posterior,5),file="ldak2.txt",quote=F,row.names=F,col.names=F)
write.table(round(ldak3$posterior,5),file="ldak3.txt",quote=F,row.names=F,col.names=F)
write.table(round(ldak4$posterior,5),file="ldak4.txt",quote=F,row.names=F,col.names=F)
write.table(round(ldak5$posterior,5),file="ldak5.txt",quote=F,row.names=F,col.names=F)
write.table(round(ldak6$posterior,5),file="ldak6.txt",quote=F,row.names=F,col.names=F)                                      
write.table(round(ldak7$posterior,5),file="ldak7.txt",quote=F,row.names=F,col.names=F)
write.table(round(ldak8$posterior,5),file="ldak8.txt",quote=F,row.names=F,col.names=F)
write.table(round(ldak9$posterior,5),file="ldak9.txt",quote=F,row.names=F,col.names=F)
write.table(round(ldak10$posterior,5),file="ldak10.txt",quote=F,row.names=F,col.names=F)

In [18]:
%%R
######### create entropy header ####

Sp_Pop_list <- paste('PR',Pop_ID$Pop,sep='_')
Pop_ID_list <- paste(Pop_ID$Pop,Pop_ID$ID,sep='_')

Header <- data.frame(dims = NA, Sp_Pop_list,Pop_ID_list)

dim(g)

[1] 38103   364


In [19]:
%%R
df <- t(Header)
dims <- paste(dim(g)[2],dim(g)[1],"1",sep = " ")
df[1,1] <- dims
df
write.table(df,'entropy_header.txt',sep = " ",na ="",
            quote = FALSE,row.names = FALSE,col.names = FALSE)


In [20]:
!head entropy_header.txt

364 38103 1                                                                                                                                                                                                                                                                                                                                                                           
PR_AN PR_AN PR_AN PR_AN PR_AN PR_AN PR_AN PR_AN PR_AN PR_AN PR_AN PR_AN PR_AS PR_AS PR_AS PR_AS PR_AS PR_AS PR_AS PR_AS PR_AS PR_AS PR_AS PR_BB PR_BB PR_BB PR_BB PR_BB PR_BB PR_BB PR_BB PR_BB PR_BB PR_BB PR_BB PR_BH PR_BH PR_BH PR_BH PR_BH PR_BH PR_BH PR_BH PR_BH PR_BH PR_BH PR_BH PR_CC PR_CC PR_CC PR_CC PR_CC PR_CC PR_CC PR_CC PR_CC PR_CC PR_CC PR_CC PR_CF PR_CF PR_CF PR_CF PR_CF PR_CF PR_CF PR_CF PR_CF PR_CL PR_CL PR_CL PR_CL PR_CL PR_CL PR_CL PR_CL PR_CL PR_CL PR_CL PR_CL PR_CN PR_CN PR_CN PR_CN PR_CN PR_CN PR_CN PR_CN PR_CN PR_CN PR_CN PR_CN PR_EC PR_EC PR_EC PR_EC PR_EC PR_EC PR_EC PR_EC PR_EC PR_EC PR_EC PR_EC P

In [21]:
!cat entropy_header.txt good_snps.recode.mpgl > entropy.mpgl

In [22]:
cd $ent_dir

/working/tfaske/balsam/entropy


In [23]:
l = 100000
b = 30000
num_k = [2,3,4,5,6]

In [24]:
def write_entropy_sh(l,b,num_k):
    with open("run_entropy.sh", "w") as o:
        o.write("""#!/bin/bash
#$ -S /bin/bash
#$ -cwd
#$ -o entropy.out
#$ -e entropy.err
module load entropy/1.2\n\n\n""")
        
#entropy -i entropy.mpgl -o entropy_k3.hdf5 \
#-l 70000 -b 30000 -t 10 -s 20 -e .01 -k 3 -q ldak3.txt -m 1 -w 0 &> k3stdout.txt &
#disown -h 
        for k in num_k:
            o.write("""entropy -i entropy.mpgl -o entropy_k{2}.hdf5 -l {0} -b {1} -t 10 -s 20 -e .01 -k {2} -q ldak{2}.txt -m 1 -w 0 &> k{2}stdout.txt &
disown -h \n\n""".format(l,b,k))

In [25]:
write_entropy_sh(l,b,num_k)

# Use estpost to get DIC, q, and gprob

In [62]:
cd $ent_dir

/working/tfaske/piper/retic/entropy


In [67]:
hdf5_files = !find . -name '*hdf5'
hdf5_files = hdf5_files.sort()
hdf5_files

['./entropy_k2.hdf5',
 './entropy_k3.hdf5',
 './entropy_k4.hdf5',
 './entropy_k5.hdf5',
 './entropy_k6.hdf5']

In [73]:
!module load entropy/1.2

In [72]:
num_k = [2,3,4,5,6]

# nvm, figure out later. Run in terminal

In [71]:
#make DIC
for i in range(0,len(hdf5_files)):
    k = num_k[i]
    f = hdf5_files[i]
    dic = "DIC_k%d.txt" % (k)
    !estpost.entropy $f -s 3 -p deviance > $dic


/usr/bin/sh: estpost.entropy: command not found
/usr/bin/sh: estpost.entropy: command not found
/usr/bin/sh: estpost.entropy: command not found
/usr/bin/sh: estpost.entropy: command not found
/usr/bin/sh: estpost.entropy: command not found


In [77]:
dic_files = !find . -name 'DIC*'
dic_files

['./DIC_k2.txt',
 './DIC_k3.txt',
 './DIC_k4.txt',
 './DIC_k5.txt',
 './DIC_k6.txt']

In [79]:
for d in dic_files:
    !cat $d
    print('\n')

file = entropy_k2.hdf5
parameter dimensions for deviance: samples = 7000, chains = 1
Model deviance: 3770153.62
Effective number of parameters: 1724916.16
Model DIC: 5495069.78


file = entropy_k3.hdf5
parameter dimensions for deviance: samples = 7000, chains = 1
Model deviance: 3691381.34
Effective number of parameters: 1891442.59
Model DIC: 5582823.93


file = entropy_k4.hdf5
parameter dimensions for deviance: samples = 7000, chains = 1
Model deviance: 3609734.37
Effective number of parameters: 2097108.35
Model DIC: 5706842.72


file = entropy_k5.hdf5
parameter dimensions for deviance: samples = 7000, chains = 1
Model deviance: 3545331.26
Effective number of parameters: 2393933.78
Model DIC: 5939265.04


file = entropy_k6.hdf5
parameter dimensions for deviance: samples = 7000, chains = 1
Model deviance: 3416931.82
Effective number of parameters: 211269059.98
Model DIC: 214685991.80


